In [1]:
import dataclasses
import requests
import logging
import base64
import pandas as pd
import openai
import os
from dotenv import load_dotenv
from tqdm import tqdm

import sys
sys.path.append('../')
import nature_go_client

from summary_generation import generate_summaries_three_paragraphs, generate_summaries_variable_length

logger = logging.getLogger(__name__)

BASE_URL = 'http://nature-go.edouardleurent.com'
LOGIN_URL = BASE_URL + '/api/auth/login/'
SPECIES_LIST_URL = BASE_URL + '/api/species/all/'
SPECIES_DETAIL_URL = BASE_URL + '/api/species/{species_id}/'
QUESTION_CREATE_URL = BASE_URL + '/api/university/quiz/questions/'

load_dotenv()

# Your id here
NG_USERNAME = os.getenv("NG_USERNAME")
NG_PASSWORD = os.getenv("NG_PASSWORD")

openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
client = nature_go_client.NatureGoClient(username=NG_USERNAME, password=NG_PASSWORD)
client.login()

In [18]:
species_list = client.get_all_species(limit=100)

species_without_summary = [e for e in species_list if e['descriptions'] == []]

In [19]:
for species in tqdm(species_without_summary):
    species_name, species_id = species['scientificNameWithoutAuthor'], species['id']
    summaries = generate_summaries(species_name)
    client.post_species_descriptions(species_id, summaries)

 28%|██▊       | 25/90 [06:28<17:30, 16.17s/it]

API error occurred: This model's maximum context length is 4097 tokens. However, your messages resulted in 6249 tokens. Please reduce the length of the messages. . Retrying in 1 second...
API error occurred: This model's maximum context length is 4097 tokens. However, your messages resulted in 6249 tokens. Please reduce the length of the messages. . Retrying in 1 second...
API error occurred: This model's maximum context length is 4097 tokens. However, you requested 4346 tokens (3546 in the messages, 800 in the completion). Please reduce the length of the messages or completion. . Retrying in 1 second...
API error occurred: This model's maximum context length is 4097 tokens. However, you requested 4346 tokens (3546 in the messages, 800 in the completion). Please reduce the length of the messages or completion. . Retrying in 1 second...
API error occurred: This model's maximum context length is 4097 tokens. However, you requested 4374 tokens (3574 in the messages, 800 in the completion)

 32%|███▏      | 29/90 [07:52<18:41, 18.38s/it]

API error occurred: This model's maximum context length is 4097 tokens. However, your messages resulted in 4335 tokens. Please reduce the length of the messages. . Retrying in 1 second...
API error occurred: This model's maximum context length is 4097 tokens. However, your messages resulted in 4335 tokens. Please reduce the length of the messages. . Retrying in 1 second...


 37%|███▋      | 33/90 [08:53<14:54, 15.70s/it]

...

 88%|████████▊ | 79/90 [21:17<02:46, 15.13s/it]

API error occurred: This model's maximum context length is 4097 tokens. However, your messages resulted in 4098 tokens. Please reduce the length of the messages. . Retrying in 1 second...
API error occurred: This model's maximum context length is 4097 tokens. However, your messages resulted in 4103 tokens. Please reduce the length of the messages. . Retrying in 1 second...


 96%|█████████▌| 86/90 [23:05<00:57, 14.49s/it]

API error occurred: This model's maximum context length is 4097 tokens. However, you requested 4724 tokens (3924 in the messages, 800 in the completion). Please reduce the length of the messages or completion. . Retrying in 1 second...
API error occurred: This model's maximum context length is 4097 tokens. However, you requested 4619 tokens (3819 in the messages, 800 in the completion). Please reduce the length of the messages or completion. . Retrying in 1 second...
API error occurred: This model's maximum context length is 4097 tokens. However, you requested 4396 tokens (3596 in the messages, 800 in the completion). Please reduce the length of the messages or completion. . Retrying in 1 second...
API error occurred: This model's maximum context length is 4097 tokens. However, you requested 4380 tokens (3580 in the messages, 800 in the completion). Please reduce the length of the messages or completion. . Retrying in 1 second...


100%|██████████| 90/90 [24:11<00:00, 16.12s/it]


## New generation 

In [3]:
import pandas as pd

df = pd.DataFrame(client.get_labeled_species(limit=5000, descriptions=False, ordering='rarity_gpt'))

print(df['wikipedia_word_count'].sum(), "total words")
print(df['wikipedia_word_count'].sum() / 1000 * 0.002, "total wiki cost")

1649616.0 total words
3.299232 total wiki cost


In [ ]:
for species in tqdm(species_without_summary):
    species_name, species_id = species['scientificNameWithoutAuthor'], species['id']
    summaries = generate_summaries(species_name)

In [24]:
df.keys()

Index(['id', 'scientificNameWithoutAuthor', 'commonNames', 'genus', 'family',
       'gbif_id', 'powo_id', 'wikipedia_word_count', 'number_of_occurrences',
       'occurences_cdf', 'rarity_gpt', 'descriptions', 'illustration',
       'illustration_transparent', 'illustration_reference',
       'illustration_reference_transparent', 'reference_image_url',
       'illustration_url', 'display_name', 'num_observations_total',
       'num_questions_total'],
      dtype='object')

In [36]:
for idx, species in df.head(200).iterrows():
    if species['wikipedia_word_count'] == -1 and species['rarity_gpt'] >= 1:
        print(species['scientificNameWithoutAuthor'], species['id'], species['rarity_gpt'], species['wikipedia_word_count'])

Cosmos peucedanifolius 24628 1.4 -1.0
Agapanthus campanulatus 30351 1.4 -1.0
Lamium flexuosum 2734 1.6 -1.0
Lamium garganicum 4428 1.6 -1.0
Lamium moschatum 7805 1.6 -1.0
Anthurium spathiphyllum 19772 1.6 -1.0
Casuarina teres 29495 1.6 -1.0
Lamium confertum 30937 1.6 -1.0
Stenotaphrum dimidiatum 8981 1.8 -1.0
Peperomia syringifolia 24012 1.8 -1.0
Peperomia tricolor 24048 1.8 -1.0
Mentha × rotundifolia 30639 1.8 -1.0
Mentha × dumetorum 30642 1.8 -1.0


In [6]:
summaries = generate_summaries_variable_length("Ocimum basilicum")

In [5]:
summaries

{'long_summary': "[DATE], I made a fascinating discovery in my botanical expedition through the treacherous wilderness. I stumbled upon a remarkable herb known as Ocimum basilicum, or basil for short. This culinary herb belongs to the mint family and is widely used in cuisines around the world. I found that basil comes in various varieties, including sweet basil, Thai basil, and lemon basil. It has a rich green color and beautiful ovate leaves of different sizes and shapes. The plant can reach impressive heights and produces small white flowers. I also learned that basil is native to tropical regions and is commonly grown as an annual plant in temperate climates. This herb has such a distinct scent due to its volatile aromatic compounds, and it is a delight to the senses. I can't wait to learn more about this fascinating plant.",
 'medium_summary': "What a thrilling day! I stumbled upon a captivating herb called basil during my dangerous expedition. This culinary herb is part of the mi

In [27]:
def replace_today(summary):
    words = summary.split(" ", 1)
    if words[0] == "Today,":
        return "[DATE], " + words[1]
    else:
        return summary
    
updated_summaries = {key: replace_today(value) for key, value in summaries.items()}
updated_summaries

{'long_summary': '[DATE], I had the pleasure of observing the fascinating Ocimum basilicum, commonly known as basil. This culinary herb, belonging to the Lamiaceae family, is a tender plant that is widely used in cuisines all over the world. It is native to tropical regions, stretching from Central Africa to Southeast Asia. Basil comes in various varieties, including sweet basil, Thai basil, and lemon basil. The leaves of this herb are richly green and ovate, with sizes ranging from 3 to 11 cm long. Basil plants produce small white flowers that develop into round achenes. The essential oil extracted from basil contains aromatic compounds like linalool and methyl chavicol, giving it a distinct scent. Basil is cultivated in many countries, thriving in hot and dry conditions. It can be grown both outdoors and indoors, and its leaves are commonly used fresh in recipes. The seeds of basil are also used in Asian drinks and desserts. Basil has cultural and religious significance in various tr

In [4]:
summaries = generate_summaries_three_paragraphs("Ocimum basilicum")

First paragraph:During my botanical expedition, I had the pleasure of encountering the marvelous Ocimum basilicum, commonly known as basil. This culinary herb belongs to the Lamiaceae family and is native to tropical regions from Central Africa to Southeast Asia. Basil is a tender plant with rich green leaves that come in various sizes and shapes, depending on the cultivar. It can reach heights of up to 150 cm and produces small white flowers. One interesting fact about basil is that it is often referred to as the "royal herb" due to its association with the production of royal perfumes. In French, it is even called "l'herbe royale."

Second paragraph:
The distinctive scents of different basil varieties are due to the varying volatile aromatic compounds they contain. For example, European basil has high concentrations of linalool and methyl chavicol, giving it a unique fragrance. Sweet basil, on the other hand, has a clove scent derived from eugenol. Basil is widely cultivated around t

In [5]:
summaries

{'first_paragraph': 'During my botanical expedition, I had the pleasure of encountering the marvelous Ocimum basilicum, commonly known as basil. This culinary herb belongs to the Lamiaceae family and is native to tropical regions from Central Africa to Southeast Asia. Basil is a tender plant with rich green leaves that come in various sizes and shapes, depending on the cultivar. It can reach heights of up to 150 cm and produces small white flowers. One interesting fact about basil is that it is often referred to as the "royal herb" due to its association with the production of royal perfumes. In French, it is even called "l\'herbe royale."',
 'second_paragraph': 'The distinctive scents of different basil varieties are due to the varying volatile aromatic compounds they contain. For example, European basil has high concentrations of linalool and methyl chavicol, giving it a unique fragrance. Sweet basil, on the other hand, has a clove scent derived from eugenol. Basil is widely cultivat

In [6]:
summaries = generate_summaries_three_paragraphs("Tradescantia zebrina")

First paragraph:Today, dear diary, I had the pleasure of observing the fascinating Tradescantia zebrina, also known as the silver inch plant or wandering Jew. This creeping plant is a true delight to behold, with its zebra-patterned leaves that display a stunning combination of purple, green, and silver. It's no wonder this plant is so popular in cultivation, as it grows rapidly and its foliage is truly attractive. I must say, the name "wandering Jew" is quite controversial, and some even prefer to call it the "wandering dude" instead. 

Second paragraph:
As I continued my observations, I noticed that the leaves of Tradescantia zebrina have a unique structure. They are bluish green in color and usually feature two longitudinal stripes that are silvery on the surface and purple on the underside. However, when exposed to intense sunlight for long periods of time, the variegation fades and the leaves turn a beautiful shade of purple throughout. The leaf sheaths are thin and translucent, a

In [7]:
summaries

{'first_paragraph': '[DATE], dear diary, I had the pleasure of observing the fascinating Tradescantia zebrina, also known as the silver inch plant or wandering Jew. This creeping plant is a true delight to behold, with its zebra-patterned leaves that display a stunning combination of purple, green, and silver. It\'s no wonder this plant is so popular in cultivation, as it grows rapidly and its foliage is truly attractive. I must say, the name "wandering Jew" is quite controversial, and some even prefer to call it the "wandering dude" instead.',
 'second_paragraph': 'As I continued my observations, I noticed that the leaves of Tradescantia zebrina have a unique structure. They are bluish green in color and usually feature two longitudinal stripes that are silvery on the surface and purple on the underside. However, when exposed to intense sunlight for long periods of time, the variegation fades and the leaves turn a beautiful shade of purple throughout. The leaf sheaths are thin and tra

In [4]:
summaries = generate_summaries_three_paragraphs("Cosmos peucedanifolius")

First paragraph:My dear diary,

Today, I stumbled upon a most intriguing plant during my botanical expedition in the vast wilderness. It goes by the name of Cosmos peucedanifolius, and what a spectacle it is to behold! Its slender stems reach for the sky, adorned with delicate fern-like leaves that seem to dance in the gentle breeze. The vibrant pink and white flowers that grace this plant are truly a sight to behold, attracting a plethora of butterflies and bees with their sweet nectar.

Ah, but the wonders of Cosmos peucedanifolius do not end there! As I carefully examined the flowers, I discovered a most fascinating fact. Each individual flower possesses both male and female reproductive parts, a trait known as perfect flowers. This clever adaptation ensures that the plant can self-pollinate, allowing it to thrive even in the most remote and isolated locations. Truly, nature has bestowed upon Cosmos peucedanifolius a remarkable survival strategy.

But wait, dear diary, there is more

In [42]:
# Function to classify rarity based on conditions
def classify_rarity(row):
    rgpt, ocdf = row['rarity_gpt'], row['occurences_cdf']

    if ocdf > 0:
            if rgpt <= 2.8:
                return 'Very Common'
    
            if 2.8 < rgpt <= 3.4 or 0.3097 <= ocdf:
                return 'Common'
    
            if 3.4 < rgpt <= 3.6 or 0.3097 > ocdf >= 0.11:
                return 'Uncommon'
    
            if 3.6 < rgpt <= 4.2 or 0.11 > ocdf >= 0.01:
                return 'Rare'
    

    if rgpt <= 2.8:
        return 'Very Common'
    
    if 2.8 < rgpt <= 3.4:
        return 'Common'
    
    if 3.4 < rgpt <= 3.8:
        return 'Uncommon'
    
    if 3.8 < rgpt <= 4.6:
        return 'Rare'
    
    if 4.6 < rgpt:
        return 'Legendary'
    
# Apply function to DataFrame and store result in a new column
df['Rarity'] = df.apply(classify_rarity, axis=1)

In [48]:
df['Rarity'].value_counts()

Rarity
Common         1964
Uncommon       1471
Very Common    1264
Rare            288
Name: count, dtype: int64